<a href="https://colab.research.google.com/github/malikumarali/malikumarali/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fsspec==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platfo

In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn scipy
!pip install shap lime
!pip install datasets transformers spacy nltk rouge-score
!python -m spacy download en_core_web_sm
import nltk
nltk.download('stopwords')

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from datasets import load_dataset

try:
    cnn_dailymail = load_dataset("cnn_dailymail", '3.0.0') # Specify the version
    print(cnn_dailymail)
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Make sure you have internet access and the 'datasets' library installed.")
    print("You can install it using: !pip install datasets")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
if 'cnn_dailymail' in locals():
    train_data = cnn_dailymail['train']
    print(train_data[0]) # Example of a training instance

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    text = text.lower()
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return " ".join(tokens)

if 'cnn_dailymail' in locals():
    example_article = train_data[0]['article']
    preprocessed_article = preprocess_text(example_article)
    print("Original Article (excerpt):", example_article[:300] + "...")
    print("Preprocessed Article (excerpt):", preprocessed_article[:300] + "...")

Original Article (excerpt): LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappoi...
Preprocessed Article (excerpt): london england reuters harry potter star daniel radcliffe gains access reported £ 20 million $ 41.1 million fortune turns 18 monday insists money wo cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnists world young actor says plans fritter cash away fas...


In [ ]:
from spacy.lang.en import English

def extractive_summarization_spacy(text, num_sentences=3):
    nlp_ext = English()
    nlp_ext.add_pipe("sentencizer")
    doc = nlp_ext(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    word_frequency = {}
    for word in nlp(text):
        if not word.is_stop and not word.is_punct and not word.is_space:
            word_frequency[word.text] = word_frequency.get(word.text, 0) + 1

    if not word_frequency:
        return "No summary could be generated."

    max_frequency = max(word_frequency.values())
    for word in word_frequency:
        word_frequency[word] = word_frequency[word] / max_frequency

    sentence_scores = {}
    for sent in sentences:
        for word in nlp(sent):
            if word.text in word_frequency:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_frequency[word.text]

    import heapq
    best_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return " ".join(best_sentences)

if 'cnn_dailymail' in locals():
    example_article = train_data[0]['article']
    extractive_summary = extractive_summarization_spacy(example_article, num_sentences=3)
    print("\nExtractive Summary (spaCy):", extractive_summary)


Extractive Summary (spaCy): Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. " LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart.


In [ ]:
from transformers import pipeline

try:
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn") # A popular choice for summarization
    if 'cnn_dailymail' in locals():
        example_article = train_data[0]['article']
        abstractive_summary = summarizer(example_article, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
        print("\nAbstractive Summary (BART):", abstractive_summary)
except Exception as e:
    print(f"Error loading summarization pipeline: {e}")
    print("Make sure you have the 'transformers' library installed.")
    print("You can install it using: !pip install transformers")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



Abstractive Summary (BART): Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund.


In [ ]:
!pip install --upgrade evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import evaluate

rouge_metric = evaluate.load("rouge")

def evaluate_summary(model, article, reference_summary):
    summary = model(article, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    results = rouge_metric.compute(predictions=[summary], references=[reference_summary])
    return summary, results

if 'cnn_dailymail' in locals():
    # Use the pre-trained BART model for a quick evaluation example
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    example_article = cnn_dailymail['test'][0]['article'] # Using test split for evaluation
    reference_summary = cnn_dailymail['test'][0]['highlights']
    generated_summary, rouge_scores = evaluate_summary(summarizer, example_article, reference_summary)

    print("\nOriginal Article (excerpt):", example_article[:500] + "...")
    print("Reference Summary:", reference_summary)
    print("Generated Summary (BART):", generated_summary)
    print("ROUGE Scores:", rouge_scores)

Device set to use cpu



Original Article (excerpt): (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin...
Reference Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated Summary (BART): The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the